In [1]:
import xarray as xr
from colorcet import bmw, coolwarm

from xpublish.routers import xyz_router

In [2]:
ds = xr.tutorial.open_dataset(
    "air_temperature", chunks=dict(lat=5, lon=5),
)
ds

<xarray.Dataset>
Dimensions:  (lat: 25, lon: 53, time: 2920)
Coordinates:
  * lat      (lat) float32 75.0 72.5 70.0 67.5 65.0 ... 25.0 22.5 20.0 17.5 15.0
  * lon      (lon) float32 200.0 202.5 205.0 207.5 ... 322.5 325.0 327.5 330.0
  * time     (time) datetime64[ns] 2013-01-01 ... 2014-12-31T18:00:00
Data variables:
    air      (time, lat, lon) float32 dask.array<chunksize=(2920, 5, 5), meta=np.ndarray>
Attributes:
    Conventions:  COARDS
    title:        4x daily NMC reanalysis (1948)
    description:  Data is from NMC initialized reanalysis\n(4x/day).  These a...
    platform:     Model
    references:   http://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanaly...

The xarray dataset should be a 2D array map.

In [3]:
ds = ds.isel(time=1)

XYZ router accepts only spatial dimensions that are named 'x' and  'y'

In [4]:
ds = ds.rename({"lat": "y", "lon": "x"})

XYZ uses morecantile to tile the dataarray. Morecantile supports different grids, not only the Web Mercator which is the most common and it is used by Google Maps and OpenStreetMap.
In this example the dataset needs to be reprojected to match the Web Mercator projection.

In [5]:
ds.rio.set_spatial_dims(x_dim="x", y_dim="y", inplace=True)
ds.rio.write_crs(4326, inplace=True)

ds = ds.rio.reproject(
    # epsg: 3857
    dst_crs= "+proj=merc +a=6378137 +b=6378137 +lat_ts=0.0 +lon_0=0.0 +x_0=0.0 +y_0=0 +k=1.0 +units=m +nadgrids=@null +wktext  +no_defs" 
    )

XYZ router accepts 2 map options to:
- crs_epsg: set the correct grid tiling 
- datashader_settings: produce custom images setting the datashader parameters

In [6]:
crs_epsg =  3857

datashader_settings = {
    # parameters for datashader.Canvas.raster method
    "raster": {"upsample_method": "nearest"}, 
    # parameters for datashader.transfer_functions.shade
    "shade": {  "cmap": ["blue","red"], 
                "how": "log", 
                "span": [float(ds["air"].min()), float(ds["air"].max())], 
                "alpha": 200 }, 
}

xyz_router.map_options(crs_epsg, datashader_settings=datashader_settings)

In [7]:
ds.rest(routers=[xyz_router])

In [8]:
import nest_asyncio 
nest_asyncio.apply()

In [ ]:
ds.rest.serve()

INFO:     Started server process [5467]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:9000 (Press CTRL+C to quit)


INFO:     127.0.0.1:39002 - "GET /tiles/air/2/2/0 HTTP/1.1" 500 Internal Server Error


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/perm/ma/maif/.local/anaconda3/envs/xpub_dev/lib/python3.8/site-packages/uvicorn/protocols/http/h11_impl.py", line 369, in run_asgi
    result = await app(self.scope, self.receive, self.send)
  File "/perm/ma/maif/.local/anaconda3/envs/xpub_dev/lib/python3.8/site-packages/uvicorn/middleware/proxy_headers.py", line 59, in __call__
    return await self.app(scope, receive, send)
  File "/perm/ma/maif/.local/anaconda3/envs/xpub_dev/lib/python3.8/site-packages/fastapi/applications.py", line 199, in __call__
    await super().__call__(scope, receive, send)
  File "/perm/ma/maif/.local/anaconda3/envs/xpub_dev/lib/python3.8/site-packages/starlette/applications.py", line 112, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/perm/ma/maif/.local/anaconda3/envs/xpub_dev/lib/python3.8/site-packages/starlette/middleware/errors.py", line 181, in __call__
    raise exc from None
  File "/perm/

INFO:     127.0.0.1:39004 - "GET /tiles/air/2/2/1 HTTP/1.1" 500 Internal Server Error


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/perm/ma/maif/.local/anaconda3/envs/xpub_dev/lib/python3.8/site-packages/uvicorn/protocols/http/h11_impl.py", line 369, in run_asgi
    result = await app(self.scope, self.receive, self.send)
  File "/perm/ma/maif/.local/anaconda3/envs/xpub_dev/lib/python3.8/site-packages/uvicorn/middleware/proxy_headers.py", line 59, in __call__
    return await self.app(scope, receive, send)
  File "/perm/ma/maif/.local/anaconda3/envs/xpub_dev/lib/python3.8/site-packages/fastapi/applications.py", line 199, in __call__
    await super().__call__(scope, receive, send)
  File "/perm/ma/maif/.local/anaconda3/envs/xpub_dev/lib/python3.8/site-packages/starlette/applications.py", line 112, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/perm/ma/maif/.local/anaconda3/envs/xpub_dev/lib/python3.8/site-packages/starlette/middleware/errors.py", line 181, in __call__
    raise exc from None
  File "/perm/

...

INFO:     127.0.0.1:39030 - "GET /tiles/air/4/3/4 HTTP/1.1" 200 OK
INFO:     127.0.0.1:39032 - "GET /tiles/air/4/5/4 HTTP/1.1" 200 OK
INFO:     127.0.0.1:39034 - "GET /tiles/air/4/3/5 HTTP/1.1" 200 OK
INFO:     127.0.0.1:39036 - "GET /tiles/air/4/6/5 HTTP/1.1" 200 OK
INFO:     127.0.0.1:39038 - "GET /tiles/air/4/6/4 HTTP/1.1" 200 OK
INFO:     127.0.0.1:39040 - "GET /tiles/air/4/7/5 HTTP/1.1" 500 Internal Server Error


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/perm/ma/maif/.local/anaconda3/envs/xpub_dev/lib/python3.8/site-packages/uvicorn/protocols/http/h11_impl.py", line 369, in run_asgi
    result = await app(self.scope, self.receive, self.send)
  File "/perm/ma/maif/.local/anaconda3/envs/xpub_dev/lib/python3.8/site-packages/uvicorn/middleware/proxy_headers.py", line 59, in __call__
    return await self.app(scope, receive, send)
  File "/perm/ma/maif/.local/anaconda3/envs/xpub_dev/lib/python3.8/site-packages/fastapi/applications.py", line 199, in __call__
    await super().__call__(scope, receive, send)
  File "/perm/ma/maif/.local/anaconda3/envs/xpub_dev/lib/python3.8/site-packages/starlette/applications.py", line 112, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/perm/ma/maif/.local/anaconda3/envs/xpub_dev/lib/python3.8/site-packages/starlette/middleware/errors.py", line 181, in __call__
    raise exc from None
  File "/perm/

INFO:     127.0.0.1:39032 - "GET /tiles/air/4/4/4 HTTP/1.1" 200 OK
INFO:     127.0.0.1:39030 - "GET /tiles/air/4/5/5 HTTP/1.1" 200 OK
INFO:     127.0.0.1:39034 - "GET /tiles/air/4/7/4 HTTP/1.1" 500 Internal Server Error


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/perm/ma/maif/.local/anaconda3/envs/xpub_dev/lib/python3.8/site-packages/uvicorn/protocols/http/h11_impl.py", line 369, in run_asgi
    result = await app(self.scope, self.receive, self.send)
  File "/perm/ma/maif/.local/anaconda3/envs/xpub_dev/lib/python3.8/site-packages/uvicorn/middleware/proxy_headers.py", line 59, in __call__
    return await self.app(scope, receive, send)
  File "/perm/ma/maif/.local/anaconda3/envs/xpub_dev/lib/python3.8/site-packages/fastapi/applications.py", line 199, in __call__
    await super().__call__(scope, receive, send)
  File "/perm/ma/maif/.local/anaconda3/envs/xpub_dev/lib/python3.8/site-packages/starlette/applications.py", line 112, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/perm/ma/maif/.local/anaconda3/envs/xpub_dev/lib/python3.8/site-packages/starlette/middleware/errors.py", line 181, in __call__
    raise exc from None
  File "/perm/

INFO:     127.0.0.1:39036 - "GET /tiles/air/4/4/5 HTTP/1.1" 200 OK
INFO:     127.0.0.1:39036 - "GET /tiles/air/4/2/4 HTTP/1.1" 200 OK
INFO:     127.0.0.1:39030 - "GET /tiles/air/4/2/5 HTTP/1.1" 200 OK
